### Retrieval-Augmented Generation with Wikipedia

In [1]:
import os
import openai
from gpt_helper import set_openai_api_key_from_txt,GPTchatClass,printmd
from wiki_helper import wiki_search
from util import printmd,extract_quoted_words
print ("openai version:[%s]"%(openai.__version__))

openai version:[0.27.8]


### Instantiate GPT Agent

In [2]:
set_openai_api_key_from_txt(key_path='../key/rilab_key.txt')
GPT = GPTchatClass(
    gpt_model='gpt-3.5-turbo', # 'gpt-3.5-turbo' / 'gpt-4'
    role_msg='Your are a helpful assistant summarizing infromation and answering user queries.')

OpenAI API Key Ready from [../key/rilab_key.txt].
Chat agent using [gpt-3.5-turbo] initialized with the follow role:[Your are a helpful assistant summarizing infromation and answering user queries.]


### Our RAG agent will use the following strategies
We assume that a user question is given (e.g., 'Who is the current president of South Korea?').
* Step 1. For the given question, our `GPT agent` will first generate a number of entities for searching Wikipedia.
* Step 2. Then, our `WikiBot` will provide (i.e., crawl) related information summarized with the `GPT agent` considering the user question.
* Step 3. Finally, the summarized texts and the original user question will be given to the `GPT agent` to answer. 

In [3]:
question = 'Who is the current president of South Korea?'
print ("question: %s"%(question))

question: Who is the current president of South Korea?


### Step 1. Generate entities for wiki search

In [4]:
user_msg = \
    """
    Suppose you will use Wikipedia for retrieving information. 
    Could you recommend three query words wrapped with quotation marks considering the following question?
    """ + '"' + question + '"'

In [5]:
response_content = GPT.chat(
    user_msg=user_msg,PRINT_USER_MSG=True,PRINT_GPT_OUTPUT=True,
    RESET_CHAT=True,RETURN_RESPONSE=True)

[USER_MSG]



    Suppose you will use Wikipedia for retrieving information. 
    Could you recommend three query words wrapped with quotation marks considering the following question?
    "Who is the current president of South Korea?"

[GPT_OUTPUT]


Certainly! Here are three query words you can use to search for the current president of South Korea on Wikipedia:

1. "Current president of South Korea"
2. "South Korea president"
3. "President of South Korea"

In [6]:
# Print summarized sentence with a markdown format
printmd(response_content)

Certainly! Here are three query words you can use to search for the current president of South Korea on Wikipedia:

1. "Current president of South Korea"
2. "South Korea president"
3. "President of South Korea"

In [7]:
entities = extract_quoted_words(response_content)
if len(entities) > 3: entities = entities[-3:]
print (entities)

['Current president of South Korea', 'South Korea president', 'President of South Korea']


### Step 2. Query entities to `WikiBot`

In [8]:
paragraphs_return = []
for entity in entities:
    paragraphs_return += wiki_search(entity=entity,VERBOSE=True)

entity:[Current president of South Korea] mismatched. use [President of South Korea] instead.
 We have total [358] paragraphs.
 After filtering, we have [31] and [8] paragraphs returned (k:[5] and m:[3])
entity:[South Korea president] matched.
 We have total [358] paragraphs.
 After filtering, we have [31] and [8] paragraphs returned (k:[5] and m:[3])
entity:[President of South Korea] matched.
 We have total [358] paragraphs.
 After filtering, we have [31] and [8] paragraphs returned (k:[5] and m:[3])


In [9]:
# Get the unique elements
paragraphs_unique = list(set(paragraphs_return))
print ("Number of paragraphs [%d] => unique ones [%d]"%
       (len(paragraphs_return),len(paragraphs_unique)))

Number of paragraphs [24] => unique ones [8]


In [10]:
# Now summarize each paragraph into a single sentence considering the question
summarized_sentences = []
for p_idx,p in enumerate(paragraphs_unique):
    user_msg = "You are given following question: "+question
    user_msg += "Could you summarize the following paragraph into one setence? \n "+p
    response_content = GPT.chat(
        user_msg=user_msg,PRINT_USER_MSG=False,PRINT_GPT_OUTPUT=False,
        RESET_CHAT=True,RETURN_RESPONSE=True)
    # Append summarized sentences
    summarized_sentences.append(response_content)
    # Print summarized sentence with a markdown format
    printmd(response_content)

The president of South Korea is the head of state and government, leading the State Council and acting as the commander-in-chief of the armed forces.

The creation of the Advisory Council of Elder Statesmen in South Korea, just before President Roh Tae Woo took office, was controversial and seen as a means to protect the status and position of a particular individual. However, these suspicions became irrelevant as former President Chun withdrew from politics and went into self-imposed exile.

The 1987 Constitution of South Korea allows the president to take emergency measures that may amend or abolish existing laws during a crisis, but it is uncertain if these measures can suspend parts of the Constitution itself; such measures must be approved by the National Assembly, and if not endorsed, they can be revoked. Additionally, the president is not permitted to dissolve the National Assembly, unlike in the previous 1980 Constitution.

The Provisional Government of the Republic of Korea, established in Shanghai in 1919, was recognized and succeeded by South Korea, as stated in both its original Constitution in 1948 and the current Constitution in 1988.

The paragraph describes the National Security Council and the Peaceful Unification Advisory Council in South Korea, with the former being chaired by the president and consisting of various members including the prime minister and ministers, and the latter serving as a government sounding board and providing rewards.

The current president of South Korea is directly elected to a five-year term with no possibility of re-election and is exempt from criminal liability.

The current president of South Korea is prohibited from seeking re-election due to a term limit set at five years since 1988, with previous terms ranging from four to seven years.

The current president of South Korea is Yoon Suk Yeol, who took office on May 10, 2022, after winning the 2022 presidential election with a narrow 48.5% of the vote.

### Step 3. Answer the question using `summarized_sentences`

In [13]:
user_msg = " ".join(summarized_sentences)
user_msg += " Using the information above, could you answer the following question? "
user_msg += question

In [15]:
response_content = GPT.chat(
    user_msg=user_msg,PRINT_USER_MSG=True,PRINT_GPT_OUTPUT=True,
    RESET_CHAT=True,RETURN_RESPONSE=True)

[USER_MSG]


The president of South Korea is the head of state and government, leading the State Council and acting as the commander-in-chief of the armed forces. The creation of the Advisory Council of Elder Statesmen in South Korea, just before President Roh Tae Woo took office, was controversial and seen as a means to protect the status and position of a particular individual. However, these suspicions became irrelevant as former President Chun withdrew from politics and went into self-imposed exile. The 1987 Constitution of South Korea allows the president to take emergency measures that may amend or abolish existing laws during a crisis, but it is uncertain if these measures can suspend parts of the Constitution itself; such measures must be approved by the National Assembly, and if not endorsed, they can be revoked. Additionally, the president is not permitted to dissolve the National Assembly, unlike in the previous 1980 Constitution. The Provisional Government of the Republic of Korea, established in Shanghai in 1919, was recognized and succeeded by South Korea, as stated in both its original Constitution in 1948 and the current Constitution in 1988. The paragraph describes the National Security Council and the Peaceful Unification Advisory Council in South Korea, with the former being chaired by the president and consisting of various members including the prime minister and ministers, and the latter serving as a government sounding board and providing rewards. The current president of South Korea is directly elected to a five-year term with no possibility of re-election and is exempt from criminal liability. The current president of South Korea is prohibited from seeking re-election due to a term limit set at five years since 1988, with previous terms ranging from four to seven years. The current president of South Korea is Yoon Suk Yeol, who took office on May 10, 2022, after winning the 2022 presidential election with a narrow 48.5% of the vote. Using the information above, could you answer the following question? Who is the current president of South Korea?

[GPT_OUTPUT]


The current president of South Korea is Yoon Suk Yeol. He took office on May 10, 2022, after winning the 2022 presidential election with a narrow 48.5% of the vote.